#Unzip MO_2018_Federally_Qualified_Health_Center_Locations.zip
#Unzip MO_pop_Sim.zip

#Install geopandas, shapely, pyproj, pyogrio

In [2]:
%%bash

#pip install shapely
#pip install pyproj
#pip install pyogrio
pip install 'geopandas[all]'

In [3]:
%%bash
head -100 Mo_pop_Sim/MO_pop_sim.csv

#The first column denotes if the residence is Urban or Rural
#The next two columns are longitude and latitude

UR,long,lat
R,-92.5062761212864,40.2792029061618
R,-92.5051318550842,40.276889612375
R,-92.5058899870041,40.2782423375098
R,-92.5198418278749,40.3277000831164
R,-92.524877366008,40.3295107045137
R,-92.5291823662756,40.3250413897606
R,-92.5324094395201,40.3249121413652
R,-92.526117330564,40.3264884161639
R,-92.5238984086017,40.3283689573053
R,-92.5249032416176,40.3271086309602
R,-92.5316566215904,40.3242213951089
R,-92.5230777749596,40.3245033699447
R,-92.5242509080343,40.3302713952819
R,-92.5237486887981,40.3242483650306
R,-92.530986265588,40.3245361361651
R,-92.5252466463363,40.3231027736977
R,-92.5219058607571,40.326671995696
R,-92.5826490381395,40.2515549341225
R,-92.5108897641274,40.2638229644672
R,-92.489846693235,40.2614590871247
R,-92.4959112103302,40.2680114210773
R,-92.5009400891376,40.2636468925299
R,-92.5039040265058,40.2688985252472
R,-92.4918817039017,40.2597362963358
R,-92.6257326039896,40.3292801694796
R,-92.6282872182202,40.3251945459803
R,-92.6234825313172,40.320922619

In [4]:
pip install geopandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [33]:
# Read the shapefile
shapefile_path = "MO_2018_Federally_Qualified_Health_Center_Locations"  # Replace with the actual path
gdf = gpd.read_file(shapefile_path)

# Now 'gdf' is a GeoDataFrame containing the data from the shapefile
# You can inspect it using:
#print(gdf.head())

print(gdf.crs) # print the coordinate reference system
crs = gdf.crs
gdf = gdf.set_index("OBJECTID")


print(gdf.columns)
fqhc_df = gdf['geometry']
fqhc_df



EPSG:4326
Index(['Group_Name', 'Facility', 'Address', 'City', 'County', 'State', 'Zip',
       'Phone', 'Latitude', 'Longitude', 'Loc_Code', 'geometry'],
      dtype='object')


OBJECTID
1      POINT (-90.55472 38.43597)
2      POINT (-91.13403 37.71462)
3      POINT (-92.60144 38.16025)
4      POINT (-90.45724 36.77261)
5       POINT (-94.49886 38.9629)
                  ...            
193    POINT (-91.77391 37.94594)
194     POINT (-93.29166 37.2243)
195    POINT (-91.56258 36.56534)
196    POINT (-90.23025 38.67777)
197    POINT (-92.20212 38.56633)
Name: geometry, Length: 197, dtype: geometry

In [18]:
# In Missouri it is roughly 56 miles per degree of longitude and 69 miles per degree of latitude. We'll use an approximation of 63 miles per unit for our circle.
DEGREES_PER_MILE = 1/ 63
circles_geom = gdf.buffer( 30 * DEGREES_PER_MILE)
fqhc_circles_df = gpd.GeoDataFrame(gdf, geometry=circles_geom, crs=crs)
fqhc_circles_df

C:\Users\add0203\AppData\Local\Temp\ipykernel_13136\4176864343.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  circles_geom = gdf.buffer( 30 * DEGREES_PER_MILE)


,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code,geometry
OBJECTID,,,,,,,,,,,,
1,COMTREA,COMTREA Byrnes Mill Health Center,100 Osage Executive Circle,House Springs,Jefferson,MO,63051,6367893372,38.435946,-90.554678,MAP,"POLYGON ((-90.07853 38.43597, -90.08082 38.389..."
2,Missouri Highlands Health Care,Viburnum Medical Clinic,18 Viburnum Center Road,Viburnum,Iron,MO,65566,5732445406,37.714620,-91.133983,MAP,"POLYGON ((-90.65784 37.71462, -90.66013 37.667..."
3,Central Ozarks Medical Center,Central Ozarks Medical Center At The Lake,3870 Columbia Avenue,Osage Beach,Miller,MO,65065,5733027490,38.160258,-92.601463,MAP,"POLYGON ((-92.12525 38.16025, -92.12754 38.113..."
4,Missouri Highlands Health Care,Missouri Highland Medical Clinic - Poplar Bluf...,"225 Physicians Park Drive, Suite 303",Poplar Bluff,Butler,MO,63901,5737856536,36.772568,-90.457206,MAP,"POLYGON ((-89.98105 36.77261, -89.98334 36.725..."
5,Swope Health Services,Swope Health Hickman Mills,8800 Blue Ridge Boulevard,Kansas City,Jackson,MO,64138,8163213200,38.962882,-94.498847,MAP,"POLYGON ((-94.02267 38.9629, -94.02496 38.9162..."
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Your Community Health Center,Your Community Health Center - Health Department,"200 North Main, Suite G51",Rolla,Phelps,MO,65401,5734586950,37.945925,-91.773948,MAP,"POLYGON ((-91.29772 37.94594, -91.30001 37.899..."
194,Jordan Valley Community Health Center,Jordan Valley Community Health Center - Medica...,"1443 N. Roberson, Suite 505",Springfield,Greene,MO,65802,4178511554,37.224321,-93.291591,MAP,"POLYGON ((-92.81547 37.2243, -92.81776 37.1776..."
195,Southern Missouri Community Health Center,Oregon County Community Health Center,"US Highway 63 North (RR 3, Box 3703)",Thayer,Oregon,MO,65791,4172642990,36.565336,-91.562618,MAP,"POLYGON ((-91.08639 36.56534, -91.08869 36.518..."


In [8]:
def create_point_column(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    Creates a geometry column of Point objects from longitude and latitude columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing longitude and latitude columns.
        lon_col (str): The name of the longitude column.
        lat_col (str): The name of the latitude column.
        crs (str, optional): Coordinate Reference System. Defaults to "EPSG:4326".

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame with the added geometry column.
    """

    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

In [35]:
# Read the shapefile
pop_file_path = "Mo_pop_sim/Mo_pop_Sim.csv"  # Replace with the actual path
big_pop_df = pd.read_csv(pop_file_path)
pop_df = big_pop_df.sample(frac=0.01) 

pop_gdf = create_point_column(pop_df, 'long', 'lat', crs)
pop_gdf

print(pop_gdf.columns)

Index(['UR', 'long', 'lat', 'geometry'], dtype='object')


In [28]:
len(pop_gdf)



633

In [26]:
pop_in_circles = gpd.sjoin(pop_gdf, fqhc_circles_df, predicate='within').groupby('OBJECTID').count()
pop_in_circles

,UR,long,lat,geometry,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code
OBJECTID,,,,,,,,,,,,,,,
1,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282,195282
2,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292,6292
3,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473,13473
4,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109,9109
5,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659,109659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281,14281
194,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572,45572
195,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822,4822


In [36]:
closest_fqhc = gpd.sjoin_nearest(pop_gdf, gdf)
closest_fqhc

c:\Users\add0203\AppData\Local\anaconda3\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,UR,long,lat,geometry,OBJECTID,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code
4863908,U,-90.627589,38.576099,POINT (-90.62759 38.5761),158,COMTREA,COMTREA High Ridge (Medical),324 Emerson Road,High Ridge,Jefferson,MO,63049,6366779977,38.483929,-90.504629,MAP
4241837,U,-90.864342,38.827997,POINT (-90.86434 38.828),50,Compass Health,Compass Health/Crider Health Center (Behaviora...,1032 Crosswinds Court,Wentzville,St. Charles,MO,63385,6363326000,38.803048,-90.817251,MAP
413765,U,-94.786606,39.787685,POINT (-94.78661 39.78768),113,Northwest Health Services,Family Medicine Associates,2303 Village Drive,St. Joseph,Buchanan,MO,64506,8162326818,39.787781,-94.798876,MAP
413765,U,-94.786606,39.787685,POINT (-94.78661 39.78768),123,Northwest Health Services,"Northwest Health Services, Inc. (Corporate Off...",2303 Village Drive,St. Joseph,Buchanan,MO,64506,8162718261,39.787760,-94.798875,MAP
413765,U,-94.786606,39.787685,POINT (-94.78661 39.78768),120,Northwest Health Services,Northwest Behavioral Health Services,2303 Village Drive,St. Joseph,Buchanan,MO,64506,8162324417,39.787760,-94.798875,MAP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74100,U,-91.492178,39.313758,POINT (-91.49218 39.31376),42,Clarity Healthcare,Clairty Healthcare,141 Communications Drive,Hannibal,Marion,MO,63401,5736031460,39.702223,-91.391803,MAP
2409229,U,-94.321068,38.923157,POINT (-94.32107 38.92316),145,Swope Health Services,Swope Health East,17844 E. 23rd Street South,Independence,Jackson,MO,64057,8169221000,39.078308,-94.370187,MAP
2448849,U,-94.364014,39.076138,POINT (-94.36401 39.07614),145,Swope Health Services,Swope Health East,17844 E. 23rd Street South,Independence,Jackson,MO,64057,8169221000,39.078308,-94.370187,MAP
928005,R,-94.370387,39.219020,POINT (-94.37039 39.21902),129,Samuel U. Rodgers Health Center,Samuel U. Rodgers Health Center-Clay County Fa...,800 Haines Drive,Liberty,Clay,MO,64068,8164130662,39.257031,-94.451666,MAP


In [37]:
count = closest_fqhc.groupby('OBJECTID').count()
count

,UR,long,lat,geometry,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code
OBJECTID,,,,,,,,,,,,,,,
1,299,299,299,299,299,299,299,299,299,299,299,299,299,299,299
2,171,171,171,171,171,171,171,171,171,171,171,171,171,171,171
3,393,393,393,393,393,393,393,393,393,393,393,393,393,393,393
4,138,138,138,138,138,138,138,138,138,138,138,138,138,138,138
5,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361,1361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,525,525,525,525,525,525,525,525,525,525,525,525,525,525,525
194,757,757,757,757,757,757,757,757,757,757,757,757,757,757,757
195,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116
